In [55]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
SCORE_DIR = "data/scores"

In [4]:
box_scores = os.listdir(SCORE_DIR)

In [7]:
len(box_scores)

9970

In [6]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [9]:
box_scores

['data/scores/202211070MIA.html',
 'data/scores/202212140DAL.html',
 'data/scores/201512090DAL.html',
 'data/scores/202212070TOR.html',
 'data/scores/201710180SAS.html',
 'data/scores/202104300MEM.html',
 'data/scores/201703080MIN.html',
 'data/scores/201903260DAL.html',
 'data/scores/202212090UTA.html',
 'data/scores/201511270CHO.html',
 'data/scores/202001160NOP.html',
 'data/scores/202203180PHO.html',
 'data/scores/202102190ORL.html',
 'data/scores/201611030MIL.html',
 'data/scores/201911180HOU.html',
 'data/scores/202210290BRK.html',
 'data/scores/201712020BRK.html',
 'data/scores/202111240GSW.html',
 'data/scores/201701260OKC.html',
 'data/scores/201602050ORL.html',
 'data/scores/202103030TOR.html',
 'data/scores/202211070DET.html',
 'data/scores/202103100DAL.html',
 'data/scores/202103280LAL.html',
 'data/scores/201801200NOP.html',
 'data/scores/201603110UTA.html',
 'data/scores/201711240BOS.html',
 'data/scores/202008220POR.html',
 'data/scores/201904070MEM.html',
 'data/scores/

In [10]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [26]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols

    line_score = line_score[["team", "total"]]

    return line_score


In [29]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {"id": f"box-{team}-game-{stat}"}, index_col = 0)[0]
    df = df.apply(pd.to_numeric, errors = "coerce")
    return df

In [49]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [56]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep = "first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]
        summaries.append(summary)

    summary = pd.concat(summaries, axis = 1).T

    game = pd.concat([summary, line_score], axis = 1)
    game["home"] = [0, 1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis = 1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format = "%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]

    games.append(full_game)
    
    print(f"{len(games)} / {len(box_scores)}")
    print()
print("Done")

1 / 9970

2 / 9970

3 / 9970

4 / 9970

5 / 9970

6 / 9970

7 / 9970

8 / 9970

9 / 9970

10 / 9970

11 / 9970

12 / 9970

13 / 9970

14 / 9970

15 / 9970

16 / 9970

17 / 9970

18 / 9970

19 / 9970

20 / 9970

21 / 9970

22 / 9970

23 / 9970

24 / 9970

25 / 9970

26 / 9970

27 / 9970

28 / 9970

29 / 9970

30 / 9970

31 / 9970

32 / 9970

33 / 9970

34 / 9970

35 / 9970

36 / 9970

37 / 9970

38 / 9970

39 / 9970

40 / 9970

41 / 9970

42 / 9970

43 / 9970

44 / 9970

45 / 9970

46 / 9970

47 / 9970

48 / 9970

49 / 9970

50 / 9970

51 / 9970

52 / 9970

53 / 9970

54 / 9970

55 / 9970

56 / 9970

57 / 9970

58 / 9970

59 / 9970

60 / 9970

61 / 9970

62 / 9970

63 / 9970

64 / 9970

65 / 9970

66 / 9970

67 / 9970

68 / 9970

69 / 9970

70 / 9970

71 / 9970

72 / 9970

73 / 9970

74 / 9970

75 / 9970

76 / 9970

77 / 9970

78 / 9970

79 / 9970

80 / 9970

81 / 9970

82 / 9970

83 / 9970

84 / 9970

85 / 9970

86 / 9970

87 / 9970

88 / 9970

89 / 9970

90 / 9970

91 / 9970

92 / 997

In [57]:
games_df = pd.concat(games, ignore_index = True)

In [58]:
games_df.to_csv("nba_games.csv")